In [1]:
!nvidia-smi

Wed Apr 24 18:58:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.09                 Driver Version: 546.09       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce MX230         WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   74C    P0              N/A / ERR! |      0MiB /  2048MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings


model_name = "BAAI/bge-small-en-v1.5"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

In [4]:
from langchain.vectorstores import Chroma

persist_directory = './app/storage/db/'

vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embeddings)

In [15]:
print(vectordb._collection)

name='langchain' id=UUID('8b73c01e-4beb-47a5-b3e2-36d13a310fa1') metadata=None tenant='default_tenant' database='default_database'


In [16]:
langchain = vectordb._collection
print(langchain.count())

64


In [5]:
q1 = "What is Gorilla?"

vectordb.similarity_search(q1, k=1)

[Document(page_content='Gorilla: Large Language Model Connected with\nMassive APIs\nShishir G. Patil1∗Tianjun Zhang1,∗Xin Wang2Joseph E. Gonzalez1\n1UC Berkeley2Microsoft Research\nsgp@berkeley.edu\nAbstract\nLarge Language Models (LLMs) have seen an impressive wave of advances re-\ncently, with models now excelling in a variety of tasks, such as mathematical\nreasoning and program synthesis. However, their potential to effectively use tools\nvia API calls remains unfulfilled. This is a challenging task even for today’s state-of-\nthe-art LLMs such as GPT-4, largely due to their inability to generate accurate input\narguments and their tendency to hallucinate the wrong usage of an API call. We\nrelease Gorilla, a finetuned LLaMA-based model that surpasses the performance\nof GPT-4 on writing API calls. When combined with a document retriever, Gorilla\ndemonstrates a strong capability to adapt to test-time document changes, enabling\nflexible user updates or version changes. It also sub

In [23]:
from langchain.docstore.document import Document

doc_1 =  Document(
    page_content="Wareconn is the best web platform for warranty maintenance.",
    metadata={
        "source": "wareconn.com",
        "page": 1
    }
)

doc_2 = Document(
    page_content="It is easy to navigate to the RMA request page in the Wareconn Customer Portal.",
    metadata={
        "source": "wareconn.com",
        "page": 2
    }
)

In [28]:
new_docs = [doc_1, doc_2]

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter


loader = PyPDFLoader(r"C:\Users\Dinakar\Documents\GenerativeAI\rag-enterprises\rag-langchain\backend\app\storage\data\Gorilla.pdf")
raw_document = loader.load()

text_splitter = CharacterTextSplitter(
separator="\n\n",
chunk_size=800,
chunk_overlap=100,
length_function=len,
)

documents = text_splitter.split_documents(raw_document)

In [5]:
# print(documents)

### Add to ChromaDB

In [30]:
id = vectordb.add_documents(
    new_docs,
    ids=["wareconn_1", "wareconn_2"]
)

In [6]:
vectordb.add_documents(documents) # if ids are not provided, they will be generated automatically

['15fa3181-d043-49a7-8f88-f13d8d84258b',
 '0a03e8da-2843-4986-bbaf-fbef578f3339',
 '54442d67-2118-445c-9f9f-ba86239345aa',
 'f12c04e6-0243-4506-92be-3a5782063ea9',
 'f5ada0cc-c293-4929-925b-bf49ee01ee4f',
 'c6f89efa-fce9-404c-bc72-1836989e09a3',
 '067b3fed-3679-45f9-a1ad-c89201d8589e',
 '83916e76-618e-4796-a47b-f4b14e7b5f42',
 'addf93a1-6cad-4c95-8ee8-d0481b39bfd3',
 'a2d0c70e-4e22-471a-a042-2e7e02a3badc',
 'd4d22b2b-83bd-4f20-a97b-d78f65075b89',
 '585cd1d0-748b-4671-bdea-3b49a1afeee2',
 'a982419a-5271-4967-bf1b-6ffc56d072c5',
 'f4a5eeba-e24b-487a-9659-0224d3860c55',
 '042ada6d-c928-49c0-bdaa-cfcb3619f2bd',
 '4785bbcb-70cd-4e4e-9bda-5884192accc8',
 'c699e98f-7fa5-489f-bb49-98efa5ea09d3',
 '295a5a50-4b6b-4c63-b98f-ddf4410ebfb5']

In [32]:
vectordb.similarity_search("What is the best web platform for warranty maintenance?", k=2)

[Document(page_content='Wareconn is the best web platform for warranty maintenance.', metadata={'page': 1, 'source': 'wareconn.com'}),
 Document(page_content='It is easy to navigate to the RMA request page in the Wareconn Customer Portal.', metadata={'page': 2, 'source': 'wareconn.com'})]

In [13]:
vectordb._collection.count()

1

### Update a Document

In [17]:
update_doc =  Document(
    page_content="Wareconn is the perfect web platform for warranty maintenance.",
    metadata={
        "source": "wareconn.com",
        "page": 1
    }
)

In [15]:
print(id)

print(vectordb._collection.get(ids=id))

['7']
{'ids': ['7'], 'embeddings': None, 'metadatas': [{'page': 1, 'source': 'wareconn.com'}], 'documents': ['Wareconn is the best web platform for warranty maintenance.'], 'uris': None, 'data': None}


In [18]:
vectordb.update_document(id[0], update_doc)

In [19]:
print(vectordb._collection.get(ids=id))

{'ids': ['7'], 'embeddings': None, 'metadatas': [{'page': 1, 'source': 'wareconn.com'}], 'documents': ['Wareconn is the perfect web platform for warranty maintenance.'], 'uris': None, 'data': None}


### Delete a Document

In [17]:
collection = vectordb._collection

print(collection)
print(f"No. of documents in the collection: {collection.count()}")

name='langchain' id=UUID('8b73c01e-4beb-47a5-b3e2-36d13a310fa1') metadata=None tenant='default_tenant' database='default_database'
No. of documents in the collection: 64


In [29]:
# Get all the document ids in the collection
# print(collection.get()) # # dict_keys(['ids', 'embeddings', 'documents', 'metadatas'])

# Filter the documents whose metadata contains the key 'source' that includes the string 'Gorilla'

# print(collection.filter_metadata("source", "Gorilla.pdf"))

coll = collection.get()

ids_to_delete = []

for idx in range(len(coll['ids'])):

    id = coll['ids'][idx]
    metadata = coll['metadatas'][idx]

    if 'Gorilla.pdf' in metadata['source']:
        ids_to_delete.append(id)
        
vectordb._collection.delete(ids_to_delete)

Delete of nonexisting embedding ID: 1a48c600-36c3-494c-a4de-da5e8510518a
Delete of nonexisting embedding ID: 1e59bcee-ac53-4bb8-8fbd-ac98d7b68738
Delete of nonexisting embedding ID: 28615cdc-4140-4aad-a24c-3509fa4a6632
Delete of nonexisting embedding ID: 293749da-381f-4dc8-8683-7944312e98b8
Delete of nonexisting embedding ID: 2db57753-8eed-4279-a1f1-c10640dc2c76
Delete of nonexisting embedding ID: 2fb34812-d859-416a-850c-3c3485513ccd


Delete of nonexisting embedding ID: 44212ef9-5250-49de-b3dc-28d35a5dc256
Delete of nonexisting embedding ID: 4550dcb2-519d-4b28-890b-196a632d39dd
Delete of nonexisting embedding ID: 4c55194c-ab5a-4ebf-8285-1fbc5baab20b
Delete of nonexisting embedding ID: 6c1c1553-86da-44d6-a2d6-c7967b8e04d7
Delete of nonexisting embedding ID: 6e7c934a-870e-4478-b3c3-c31fd28abd1a
Delete of nonexisting embedding ID: 7bc0b60d-dbc4-4d7b-bb88-253821d6a163
Delete of nonexisting embedding ID: 88e5325b-d57a-4a59-a096-fa48f7f2fdbc
Delete of nonexisting embedding ID: 8dad088f-6a94-4f5b-a1ef-262c94e3b26d
Delete of nonexisting embedding ID: b0c468c5-f747-4a3a-8da2-5ef2eb1a5dd1
Delete of nonexisting embedding ID: b2337f4e-d3f0-4ed3-bf99-473e4d95b735
Delete of nonexisting embedding ID: d487ad73-74c8-422d-9d77-b5cd496cd54b
Delete of nonexisting embedding ID: ec6e3483-bd35-4372-bef8-7e303720922f


In [30]:
print(collection.count())

46


In [21]:
vectordb._collection.count()

0

In [31]:
vectordb.similarity_search("What is Gorilla?", k=2)

[Document(page_content="What's \nunique\n•AI-driven feedback and \nimprovement suggestions\n•Wide range of practice modes \ntailored to different communication \nscenarios\n•Engaging Community Chat for \ncollaborative learning and resource \nsharing\n•Speech -to-text analysis for \nimmediate feedback\n•Integration of famous quotes and \nexcerpts for enriched learning \nexperience", metadata={'page': 7, 'source': 'C:\\Users\\Dinakar\\Documents\\GenerativeAI\\rag-enterprises\\rag-langchain\\backend\\app/storage/data\\Final Year Project Presentation.pdf'}),
 Document(page_content="Problems we're trying to solve\nMASTERING CHATGPT (OR \nSIMILAR LLMS) INTERACTIONS \nLEADS TO MORE EFFECTIVE \nCOMMUNICATION AND \nBETTER RESULTS FROM THE AI.\nDESCRIPTIVE WRITING SKILLS \nENHANCE MIDJOURNEY -\nGENERATED IMAGE -TO-TEXT \nRESULTS, BOOSTING \nCREATIVITY AND ACCURACY.\nCONFIDENT AND ENGAGING \nPRESENTATIONS ARE \nESSENTIAL FOR CONVEYING \nIDEAS AND SECURING \nSUPPORT.\nEFFECTIVE STORYTELLING \nSKIL